In this tutorial we are going to implement a transfer learning model for text version of the ULMfit. 
We are first running the language model needed for the classifier.

In [58]:
import os

# username 
os.environ['KAGGLE_USERNAME'] = "jlealtru" 
# key
os.environ['KAGGLE_KEY'] = "6c3a4d6b4d8e7804780d6cb02879ac53"

In [59]:
path = os.getcwd()

In [41]:
# Download from Kaggle
!kaggle datasets download -d nolanbconaway/pitchfork-data --force --unzip # api copied from kaggle

100%|██████████████████████████████████████| 34.4M/34.4M [00:17<00:00, 1.66MB/s]



In [60]:
# connect to database
import sqlite3 as lite
import pandas as pd
import numpy as np
from pandas.io.sql import read_sql

con = lite.connect('database.sqlite')

In [61]:
# grab content of the reviews
df = read_sql(""" SELECT a.content, b.* FROM content a LEFT JOIN reviews b """
              """on a.reviewid = b.reviewid """, con, coerce_float=True, params=None) 
df.head(2)

,content,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,"“Trip-hop” eventually became a ’90s punchline,...",22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,"Eight years, five albums, and two EPs in, the ...",22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017


In [62]:
df = df[['content','score', 'reviewid']]

In [65]:
# import libraries, define path, batchsize and bptt
import os
from fastai.text import *
path=os.getcwd()
bs,bptt=40,60


In [66]:
%%time
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size= 0.2, random_state= 42)

data_lm = TextLMDataBunch.from_df( path, train_df = train_df, 
                                  valid_df = val_df, text_cols = ['content'], 
                                  label_cols = ['label'], bs= bs, bptt= bptt)

CPU times: user 7.72 s, sys: 2.18 s, total: 9.9 s
Wall time: 26.3 s


In [67]:
data_lm.save('data_lm_pitchfork.pkl')

In [70]:
data_lm = load_data(path, 'data_lm_pitchfork.pkl', bs=bs, bptt=bptt)

In [72]:
learn_pitchfork = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)


In [ ]:
learn_pitchfork.lr_find()
learn_pitchfork.recorder.plot(skip_end=15)


In [73]:
#learn_pitchfork.fit_one_cycle(1, 1e-2/2, moms=(0.8,0.7))
learn_pitchfork.fit_one_cycle(1, 2e-2/2, moms=(0.8,0.7), wd=0.1)


epoch,train_loss,valid_loss,accuracy,time
0,4.618757,4.442286,0.296847,17:06


In [74]:
learn_pitchfork.save(path+'/fit_head')


In [75]:
print('unfreeze model')
learn_pitchfork.unfreeze()

unfreeze model


In [76]:
#learn.fit_one_cycle(10, 2e-3, moms=(0.8,0.7), wd=0.1)
learn_pitchfork.fit_one_cycle(12, 2e-3/3, moms=(0.8,0.7), wd= 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,4.329775,4.249726,0.315335,18:34
1,4.267435,4.192162,0.323707,19:00
2,4.291625,4.180893,0.326141,19:05
3,4.240601,4.156091,0.328409,18:44
4,4.209528,4.138834,0.331526,18:41
5,4.172590,4.115703,0.334366,18:42
6,4.094367,4.093990,0.337025,18:41
7,4.057703,4.073229,0.339719,18:42
8,4.029308,4.055165,0.342347,18:42
9,3.976991,4.040740,0.344423,18:42


In [77]:
learn_pitchfork.save(path+'/fine_tuned')
learn_pitchfork.save_encoder(path+'/fine_tuned_enc')

In [78]:
learn_pitchfork.export(path+'lang_model_pitch.pkl')

In [79]:
# save vocabulary
with open(path+'/itos.pkl', 'wb') as f:
    pickle.dump(data_lm.vocab.itos, f)